In [10]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

C:\Users\LERRY\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 数据导入与预处理

#### 数据导入

In [8]:
base_train=pd.read_csv('data/base-train.csv')
base_test=pd.read_csv('data/base-test.csv')
know_train=pd.read_csv('data/knowledge-train.csv')
know_test=pd.read_csv('data/knowledge-test.csv')
money_train=pd.read_csv('data/money-train.csv')
money_test=pd.read_csv('data/money-test.csv')
year_train=pd.read_csv('data/year-train.csv')
year_test=pd.read_csv('data/year-test.csv')

#### 数据维度

In [126]:
print('base_train:',base_train.shape)
print('know_train:',know_train.shape)
print('money_train',money_train.shape)
print('year_train',year_train.shape)

base_train: (17319, 8)
know_train: (17319, 4)
money_train (51954, 10)
year_train (51954, 11)


#### 分别按ID列排序，方便合并多个数据集

In [15]:
base_train.sort_values('ID',inplace=True)
know_train.sort_values('ID',inplace=True)
money_train.sort_values('ID',inplace=True)
year_train.sort_values('ID',inplace=True)

In [90]:
list1=[year2015,year2016]

#### 分别按年份拆分money和year数据集

In [94]:
#定义拆分数据集的方法
def split_dataset_by(df,col='year'):
    """
    df：要处理的数据集的dataframe
    col：要按哪一列处理
    return:拆分后的数据集dataframe组成的列表
    """
    split_df=[]
    col_values=df[col].value_counts().index.values#col这一列的唯一值
    columns=df.columns.values.tolist()#df所有列名
    columns.remove(col)#删除掉要拆分的这一列的名称col
    #循环拆分数据集
    for col_value in col_values:
        col_value=int(col_value)
        locals()[col+str(col_value)]=df[df[col].isin([col_value])].drop([col],axis=1)
        new_columns=['ID']
        for column in columns:
            if column=='ID':
                continue
            new_columns.append(column+str(col_value))
        locals()[col+str(col_value)].columns=new_columns#拆分出来的数据集重新命名它的列名称
        split_df.append(locals()[col+str(col_value)])#处理完成的数据集添加到列表中
    return split_df

In [95]:
split_money_train=split_dataset_by(money_train)#拆分money_train

In [124]:
split_year_train=split_dataset_by(year_train)#拆分year_train

#### 合并数据集

In [130]:
#定义合并数据集的方法
def merge_dataset(dfs,col='ID'):
    """
    df：要处理的数据集的dataframe
    col：要按哪一列处理
    return:合并后的数据集dataframe
    """
    df_res=pd.DataFrame()
    for df in dfs:
        if len(df_res)==0:
            df_res=df
        else:
            df_res=pd.merge(df_res,df,on=col,how='outer')#按col列合并，outer表示保留两个表的信息
    return df_res

In [118]:
money_train_res=merge_dataset(split_money_train)#合并money_train拆分的数据集

In [125]:
year_train_res=merge_dataset(split_year_train)#合并year_train拆分的数据集

In [127]:
#合并后的数据集维度
print('money_train_res:',money_train_res.shape)
print('year_train_res:',year_train_res.shape)

money_train_res: (17318, 25)
year_train_res: (17318, 28)


In [131]:
#合并所有数据集
train=merge_dataset([base_train,know_train,money_train_res,year_train_res])

In [132]:
#查看合并后的数据集
train

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,...,所有者权益合计2017,从业人数2015,资产总额2015,负债总额2015,营业总收入2015,主营业务收入2015,利润总额2015,净利润2015,纳税总额2015,所有者权益合计2015
0,5978034,2009.0,5600.0,工业,湖南,合伙企业,自然人,0.70,0.0,1.0,...,92400.0,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0
1,5978036,2012.0,7960.0,商业服务业,湖北,股份有限公司,自然人,0.54,0.0,0.0,...,-87560.0,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0
2,5978037,2012.0,6590.0,商业服务业,山东,农民专业合作社,企业法人,0.97,1.0,1.0,...,-6590.0,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0
3,5978038,2006.0,5650.0,商业服务业,江西,股份有限公司,自然人,0.91,0.0,0.0,...,-22600.0,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0
4,5978039,2009.0,NaN,服务业,山东,集体所有制企业,自然人,0.88,0.0,0.0,...,26775.0,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17314,5999995,2000.0,9650.0,工业,湖南,合伙企业,企业法人,0.83,0.0,1.0,...,414950.0,150.0,144750.0,202650.0,709275.0,354637.5,283710.0,70927.5,141855.0,-57900.0
17315,5999997,2013.0,9030.0,交通运输业,福建,集体所有制企业,企业法人,0.73,1.0,1.0,...,-58695.0,362.0,90300.0,81270.0,397320.0,238392.0,198660.0,79464.0,238392.0,9030.0
17316,5999998,2014.0,4510.0,服务业,湖南,股份有限公司,自然人,0.64,0.0,0.0,...,4510.0,930.0,13530.0,18040.0,4059.0,2841.3,1623.6,811.8,2435.4,-4510.0
17317,5999999,2014.0,9130.0,交通运输业,福建,股份有限公司,自然人,0.80,1.0,1.0,...,9130.0,885.0,27390.0,9130.0,60258.0,42180.6,30129.0,6025.8,24103.2,18260.0


#### 异常值处理

In [134]:
#缺失值分布统计
train.isnull().sum()

ID               0
注册时间           188
注册资本           179
行业             162
区域             163
              ... 
主营业务收入2015     327
利润总额2015       333
净利润2015        362
纳税总额2015       344
所有者权益合计2015    355
Length: 62, dtype: int64

In [136]:
#填补缺失值
train=train.fillna(method='ffill')

#### 标签编码
LabelEncoder是用来对分类型特征值进行编码，即对不连续的数值或文本进行编码。其中包含以下常用方法：  

fit(y) ：fit可看做一本空字典，y可看作要塞到字典中的词。  
fit_transform(y)：相当于先进行fit再进行transform，即把y塞到字典中去以后再进行transform得到索引值。  

In [157]:
#待编码的特征
cols_encode=['注册时间','行业','区域','企业类型','控制人类型']

In [140]:
#将数值类型（数值作为类型的列）转化为字符串类型
feature=['注册时间']
for i in feature:
    train[i]=train[i].astype(str)

In [158]:
#对不连续的数字或文本进行编号，转换成连续的数值型变量
for c in cols_encode:
    le=LabelEncoder()
    le.fit(list(train[c].values))
    train[c]=le.transform(list(train[c].values))

In [159]:
train.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,...,所有者权益合计2017,从业人数2015,资产总额2015,负债总额2015,营业总收入2015,主营业务收入2015,利润总额2015,净利润2015,纳税总额2015,所有者权益合计2015
0,0,9,5600.0,2,5,1,1,0.70,0.0,1.0,...,92400.0,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0
1,1,12,7960.0,1,4,3,1,0.54,0.0,0.0,...,-87560.0,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0
2,2,12,6590.0,1,0,0,0,0.97,1.0,1.0,...,-6590.0,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0
3,3,6,5650.0,1,3,3,1,0.91,0.0,0.0,...,-22600.0,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0
4,4,9,5650.0,3,0,4,1,0.88,0.0,0.0,...,26775.0,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0


#### 将预测列放到最后一列

In [162]:
current_columns=train.columns.drop('利润总额2017').values.tolist()
current_columns.append('利润总额2017')
current_columns

['ID',
 '注册时间',
 '注册资本',
 '行业',
 '区域',
 '企业类型',
 '控制人类型',
 '控制人持股比例',
 '专利',
 '商标',
 '著作权',
 '债权融资额度2017',
 '债权融资成本2017',
 '股权融资额度2017',
 '股权融资成本2017',
 '内部融资和贸易融资额度2017',
 '内部融资和贸易融资成本2017',
 '项目融资和政策融资额度2017',
 '项目融资和政策融资成本2017',
 '债权融资额度2016',
 '债权融资成本2016',
 '股权融资额度2016',
 '股权融资成本2016',
 '内部融资和贸易融资额度2016',
 '内部融资和贸易融资成本2016',
 '项目融资和政策融资额度2016',
 '项目融资和政策融资成本2016',
 '债权融资额度2015',
 '债权融资成本2015',
 '股权融资额度2015',
 '股权融资成本2015',
 '内部融资和贸易融资额度2015',
 '内部融资和贸易融资成本2015',
 '项目融资和政策融资额度2015',
 '项目融资和政策融资成本2015',
 '从业人数2016',
 '资产总额2016',
 '负债总额2016',
 '营业总收入2016',
 '主营业务收入2016',
 '利润总额2016',
 '净利润2016',
 '纳税总额2016',
 '所有者权益合计2016',
 '从业人数2017',
 '资产总额2017',
 '负债总额2017',
 '营业总收入2017',
 '主营业务收入2017',
 '净利润2017',
 '纳税总额2017',
 '所有者权益合计2017',
 '从业人数2015',
 '资产总额2015',
 '负债总额2015',
 '营业总收入2015',
 '主营业务收入2015',
 '利润总额2015',
 '净利润2015',
 '纳税总额2015',
 '所有者权益合计2015',
 '利润总额2017']

In [164]:
train=train.loc[:,current_columns]
train.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,...,从业人数2015,资产总额2015,负债总额2015,营业总收入2015,主营业务收入2015,利润总额2015,净利润2015,纳税总额2015,所有者权益合计2015,利润总额2017
0,0,9,5600.0,2,5,1,1,0.70,0.0,1.0,...,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0,46592.0
1,1,12,7960.0,1,4,3,1,0.54,0.0,0.0,...,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0,113430.0
2,2,12,6590.0,1,0,0,0,0.97,1.0,1.0,...,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0,14498.0
3,3,6,5650.0,1,3,3,1,0.91,0.0,0.0,...,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0,70851.0
4,4,9,5650.0,3,0,4,1,0.88,0.0,0.0,...,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0,4284.0


In [165]:
#保存处理后的数据集
train.to_csv('output/profit_forecast_train.csv',index=False)

# 定义模型

In [ ]:
def load_xy(df):
    x=df.